In [1]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, Sampler
from copy import deepcopy
from tqdm import tqdm
import sys
sys.path.append('../FakeDatasetMaking/')
from pair_nomet_creation2 import KinematicDataset, EpochSampler


In [2]:
modelsavepath='/home/ddemler/HNLclassifier/saved_files/saved_models/FNN_FeatureRegression/fnn_aug18all.pt'
pdsavepath='/home/ddemler/HNLclassifier/saved_files/fnn_featregr/fnn_aug18all.csv'

out_feats=['deltaphi', 'deltaeta', 'deltaR', 'mt', 'norm_mt', 'mass', 'pt', 'eta' , 'phi',  'px', 'py', 'pz', 'energy']
tryrel=[ 'mt', 'pt', 'px', 'py', 'pz', 'energy']
customlossindices=[idx for idx, feat in enumerate(out_feats) if feat in tryrel]

hidden_layers = [16,26,32,32,32,48,52,48,32,32,32,26]



In [3]:
train_dataset = KinematicDataset(num_events=1000000, seed=0)
input_dim, output_dim = train_dataset.usefulvariables()
# print(input_dim, output_dim)

# print(input_dim, output_dim)

train_sampler = EpochSampler(train_dataset)

train_loader = DataLoader(train_dataset, batch_size=320, sampler=train_sampler)


8 13
8 13


In [4]:
val_dataset = KinematicDataset(num_events=500000, seed=10000)
input_dim, output_dim = val_dataset.usefulvariables()

val_sampler = EpochSampler(val_dataset)

val_loader = DataLoader(val_dataset, batch_size=320, sampler=val_sampler)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class KinematicNet(nn.Module):
    def __init__(self):
        super(KinematicNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, int(input_dim*1.5))
        self.fc2 = nn.Linear(int(input_dim*1.5), int(input_dim//3))
        self.fc3 = nn.Linear(int(input_dim//3), output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


class CustomKinematicNet(nn.Module):
    def __init__(self, input_size, hidden_layers, lenoutput, activation_fn=F.relu):
        """
        Args:
        - input_size (int): Size of the input layer.
        - hidden_layers (list of int): Sizes of each hidden layer.
        - lenoutput (int): Size of the output layer.
        - activation_fn (callable): Activation function to use.
        """
        super(CustomKinematicNet, self).__init__()
        
        # Create the list of layers
        layers = [nn.Linear(input_size, hidden_layers[0])]
        for i in range(len(hidden_layers) - 1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
        layers.append(nn.Linear(hidden_layers[-1], lenoutput))
        
        self.layers = nn.ModuleList(layers)
        self.activation_fn = activation_fn
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation_fn(layer(x))
        return self.layers[-1](x)
    




def custom_loss(y_pred, y_true):
    se_loss = (y_pred - y_true) ** 2
    # print(se_loss.shape)
    num_features = int(output_dim)

    loss_list=[]

    for i in range(num_features):

        if i in customlossindices:
            RMSE=((y_pred[:,i]-y_true[:,i])/y_true[:,i])**2
            mask = (y_true[:,i] > 1)

            RMSE_meanloss=torch.mean(RMSE[mask])
            MSE_meanloss = torch.mean(se_loss[:,i][~mask])

            se_loss[mask, i] = RMSE[mask]

            loss_list.append(MSE_meanloss.item())
            loss_list.append(RMSE_meanloss.item())
        else:
            loss = torch.mean(se_loss[:,i])
            loss_list.append(loss.item())
    
    full_loss = torch.mean(se_loss)
    return loss_list, full_loss


        
        
def l2_regularization(model, lambda_reg):
    l2_reg = 0.0
    for W in model.parameters():
        l2_reg += torch.sum(W ** 2)
    return l2_reg * lambda_reg      
        



In [6]:
# hidden_layers=[64,72,82,92,102,112,122,132,142,132,122,112,102,92,82,72,64]
# hidden_layers=[64,72,82,92,102,112,122,132,142,132,102,92,82]

model = CustomKinematicNet(input_size=input_dim, hidden_layers=hidden_layers, lenoutput=output_dim)
model.to(device)

CustomKinematicNet(
  (layers): ModuleList(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=72, bias=True)
    (2): Linear(in_features=72, out_features=82, bias=True)
    (3): Linear(in_features=82, out_features=92, bias=True)
    (4): Linear(in_features=92, out_features=102, bias=True)
    (5): Linear(in_features=102, out_features=112, bias=True)
    (6): Linear(in_features=112, out_features=122, bias=True)
    (7): Linear(in_features=122, out_features=132, bias=True)
    (8): Linear(in_features=132, out_features=142, bias=True)
    (9): Linear(in_features=142, out_features=132, bias=True)
    (10): Linear(in_features=132, out_features=102, bias=True)
    (11): Linear(in_features=102, out_features=92, bias=True)
    (12): Linear(in_features=92, out_features=82, bias=True)
    (13): Linear(in_features=82, out_features=13, bias=True)
  )
)

In [7]:
out_feats=['deltaphi', 'deltaeta', 'deltaR', 'mt', 'norm_mt', 'mass', 'pt', 'eta' , 'phi',  'px', 'py', 'pz', 'energy']

df_outfeats=[]
for i, feat in enumerate(out_feats):
    if i in customlossindices:
        df_outfeats.append(feat +"_MSE")
        df_outfeats.append(feat +"_RMSE")
    else:
        df_outfeats.append(feat +"_MSE")

losses_cols=['train_loss', 'val_loss', 'l2sum']+df_outfeats

print(losses_cols)

['train_loss', 'val_loss', 'l2sum', 'deltaphi_MSE', 'deltaeta_MSE', 'deltaR_MSE', 'mt_MSE', 'mt_RMSE', 'norm_mt_MSE', 'mass_MSE', 'pt_MSE', 'pt_RMSE', 'eta_MSE', 'phi_MSE', 'px_MSE', 'px_RMSE', 'py_MSE', 'py_RMSE', 'pz_MSE', 'pz_RMSE', 'energy_MSE', 'energy_RMSE']


In [9]:
import time

# Timing data generation
start_time_generation = time.time()

# Call your data generation function/method here
train_dataset.generate_data()

end_time_generation = time.time()
generation_duration = end_time_generation - start_time_generation
print(f"Data generation took {generation_duration:.2f} seconds.")


Data generation took 54.57 seconds.


In [10]:



optimizer=torch.optim.Adam(model.parameters(), lr=0.0001)
# loss_fn=nn.MSELoss()


out_feats=['deltaphi', 'deltaeta', 'deltaR', 'mt', 'norm_mt', 'mass', 'pt', 'eta' , 'phi',  'px', 'py', 'pz', 'energy']

df_outfeats=[]
for i, feat in enumerate(out_feats):
    if i in customlossindices:
        df_outfeats.append(feat +"_MSE")
        df_outfeats.append(feat +"_RMSE")
    else:
        df_outfeats.append(feat +"_MSE")

losses_cols=['train_loss', 'val_loss', 'l2sum']+df_outfeats
losses_df=pd.DataFrame(columns=losses_cols)


numepochs=10000
best_loss=np.inf
for epoch in range(numepochs):
    model.train()
    train_loss=0
    l2sum=0
    # train_featsloss_list=[]
    for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader), leave=False, position=0, disable=True):
        x=x.to(device)
        y=y.to(device)
        y_pred=model(x)
        train_featslosssep, original_loss = custom_loss(y_pred, y)
        # train_featsloss_list.append(train_featslosssep.cpu().numpy())
        l2_loss = l2_regularization(model, lambda_reg=1e-7)
        loss = original_loss + l2_loss
        l2sum+=l2_loss.item()
        train_loss += original_loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # train_featsloss_array = np.array(train_featsloss_list)
    # avg_train_featsloss = np.mean(train_featsloss_array, axis=0)
    
    model.eval()
    # patience=20
    with torch.no_grad():
        x,y=next(iter(val_loader))
        # x=val_loader
        x=x.to(device)
        y=y.to(device)
        y_pred=model(x)
        feats_loss, valloss = custom_loss(y_pred, y)
        # valloss=sum(feats_loss)/len(feats_loss)
        

        if valloss<best_loss:
            best_loss=valloss
            patience=30
            modelsave=deepcopy(model.state_dict())
            torch.save(modelsave, modelsavepath)
        else:
            patience-=1
            if patience==0:
                print('early stopping')
                break
    # indice=[3,6,12]
    # for idx in indice:
    #     feats_loss[idx]=feats_loss[idx].cpu().float()
    
    loss_strings = [f"{df_outfeats[i]}: {feats_loss[i]:.4e}" for i in range(len(df_outfeats))]
    loss_summary = ", ".join(loss_strings)
    loss_values = [train_loss/len(train_loader), valloss.item(), l2sum]
    loss_values.extend(feats_loss)

    losses_df.loc[epoch] = loss_values
    losses_df.to_csv(pdsavepath)
    


    print(f"epoch: {epoch}, train: {train_loss/len(train_loader):.4e}, val: {valloss.item():.4e}, {loss_summary}")

    # valloss=valloss.cpu().float()
    # valloss2=valloss.item()
    # loss_strings = [f"{out_feats[i]}: {feats_loss[i]:.4e}" for i in range(len(out_feats))]
    # loss_summary = ", ".join(loss_strings)
    # loss_values = [train_loss/len(train_loader), valloss2, l2sum]
    # loss_values.extend(feats_loss)
    # # for losses in loss_values:
    #     # print(type(losses))
    # losses_df.loc[epoch] = loss_values
    # losses_df.to_csv(pdsavepath)
    # # losses_df.loc[epoch]=[train_loss/len(train_loader), valloss2, feats_loss[0], feats_loss[1]]
    # print(f"epoch: {epoch}, train: {train_loss/len(train_loader):.4e}, val: {valloss2:.4e}, {loss_summary}")


One epoch of training took 166.57 seconds.
epoch: 0, train: 2.2959e+03, val: 1.8356e+03, deltaphi_MSE: 8.0645e-01, deltaeta_MSE: 8.4590e-01, deltaR_MSE: 7.6279e-01, mt_MSE: 4.0161e+00, mt_RMSE: 8.6907e-01, norm_mt_MSE: 4.0097e-04, mass_MSE: 6.5630e+03, pt_MSE: nan, pt_RMSE: 5.4325e-01, eta_MSE: 2.4629e+00, phi_MSE: 3.4382e+00, px_MSE: 2.4007e+03, px_RMSE: 3.9028e+01, py_MSE: 2.1103e+03, py_RMSE: 3.3280e+01, pz_MSE: 2.8755e+04, pz_RMSE: 5.6844e+01, energy_MSE: nan, energy_RMSE: 2.7054e-01


KeyboardInterrupt: 

In [ ]:
modelsavedpath='/home/ddemler/HNLclassifier/fnn_FeatureRegression/fnn_aug17all_oldloss.pt'
modelsave=CustomKinematicNet(input_size=input_dim, hidden_layers=hidden_layers, lenoutput=output_dim)
modelsave.load_state_dict(torch.load(modelsavedpath))
modelsave.to(device)
out_feats=['deltaphi', 'deltaeta', 'deltaR', 'mt', 'norm_mt', 'mass', 'pt', 'eta' , 'phi',  'px', 'py', 'pz', 'energy']


test_dataset = KinematicDataset(num_events=500000, seed=10000)
input_dim, output_dim = test_dataset.usefulvariables()
test_sampler = EpochSampler(train_dataset)

test_loader = DataLoader(test_dataset, batch_size=320, sampler=test_sampler)

residuals=np.array([])
y_total=np.array([])
y_pred_total=np.array([])
for i, (x,y) in enumerate(test_loader):
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    y_pred_total=np.append(y_pred_total, y_pred.cpu().detach().numpy())
    y_total=np.append(y_total, y.cpu().detach().numpy())


    # residuals=np.append(residuals, y_pred.cpu().detach().numpy() - y.cpu().detach().numpy())

    # residuals.append(y_pred.cpu().detach().numpy() - y.cpu().detach().numpy())


numfeatures=13
y_pred_total = y_pred_total.reshape(-1,3,numfeatures)
y_total = y_total.reshape(-1,3,numfeatures)

# print(y_pred_total.shape)
# print(y_total.shape)

residuals = [[] for _ in range(numfeatures)]
label_values = [[] for _ in range(numfeatures)]

for i in range(numfeatures):
    y_curr=y_total[:,:,i]
    # print("ycurr shape before reshape", y_curr.shape)
    y_curr=y_curr.reshape(500000*3,1)
    # print("ycurr shape after reshape", y_curr.shape)
    y_pred_curr=y_pred_total[:,:,i]
    y_pred_curr=y_pred_curr.reshape(500000*3,1)
    residuals_curr = y_pred_curr - y_curr
    residuals[i]=residuals_curr
    label_values[i]=y_curr

residuals = [np.array(res_list) for res_list in residuals]  # Convert lists of arrays to arrays
# residual_medians = [np.median(res) for res in residuals]
residual_std_devs = [np.std(res) for res in residuals]
residual_means = [np.mean(res) for res in residuals]


num_rows = 4
num_cols = 5


fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 15))
flat_axes = axes.flatten()

for i, ax in enumerate(flat_axes[:numfeatures]):
    ax.hist(residuals[i], bins=100, edgecolor='k', alpha=0.65)
    ax.axvline(x=residual_means[i] + residual_std_devs[i], color='r', linestyle='--', label=f'+1 std = {residual_means[i] + residual_std_devs[i]:.2f})')
    ax.axvline(x=residual_means[i] - residual_std_devs[i], color='b', linestyle='--', label=f'-1 std = {residual_means[i] - residual_std_devs[i]:.2f})')
    ax.set_title(f'Residuals for {out_feats[i]}')
    ax.set_yscale('log')
    ax.set_xlabel('Residual Value')
    ax.set_ylabel('Frequency')
    ax.legend()
    
    # Display the mean value on the plot
    mean_text = f"Mean: {residual_means[i]:.2f}, std: {residual_std_devs[i]:.5f}"
    ax.text(0.6, 0.85, mean_text, transform=ax.transAxes)
    

for ax in flat_axes[numfeatures:]:
    ax.axis('off')

plt.tight_layout()
plt.show()
    
    


Generating data for seed: 10000


ValueError: cannot reshape array of size 3000000 into shape (1500000,1)